<a href="https://colab.research.google.com/github/jhyoo78/jhyoo78/blob/main/Copy_of_02_langchain_chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cohere openai tiktoken
!pip install -qU langchain openai


# [LangChain Handbook](https://github.com/pinecone-io/examples/tree/master/generation/langchain/handbook)

## Getting Started with Chains

Chains are the core of LangChain. They are simply a chain of components, executed in a particular order.

The simplest of these chains is the `LLMChain`. It works by taking a user's input, passing in to the first element in the chain — a `PromptTemplate` — to format the input into a particular prompt. The formatted prompt is then passed to the next (and final) element in the chain — a LLM.

We'll start by importing all the libraries that we'll be using in this example.

 Chains 는 LangChain의 핵심 기능이다. 이것은 단순히 구성 요소들을 연결하는 것이고, 특정된 순서로 실행된다.

이들 chain 중 가장 간단한 것이 LLMChain이다.
사용자의 입력을 받아 chain의 첫 번째 요소인 PromptTemplate로 전달하여 입력을 특정 Prompt로  format하여 동작한다. 그 다음 format된 Prompt는 chain의  마지막 요소인 LLM으로 전달된다.

이 예에서 사용할 모든 라이브러리를 불러오는 것으로 시작한다.

In [2]:
import inspect
import re

from getpass import getpass
from langchain import OpenAI, PromptTemplate
from langchain.chains import LLMChain, LLMMathChain, TransformChain, SequentialChain
from langchain.callbacks import get_openai_callback

To run this notebook, we will need to use an OpenAI LLM. Here we will setup the LLM we will use for the whole notebook, just input your openai api key when prompted.

이 노트북을 실행하려면 OpenAI LLM을 사용해야 한다. 여기서  사용할 LLM을 설정할 것이고, 메시지가 나타나면 openai api 키를 입력하기만 하면 된다.

In [4]:
OPENAI_API_KEY = getpass()   # 코드에서 Key 노출을 막으면서 Key를 환경 변수로 입력함.-->  입력 후 Return key 누를 것

··········


In [5]:
llm = OpenAI(
    temperature=0,     # 변경...
    openai_api_key=OPENAI_API_KEY
    )

An extra utility we will use is this function that will tell us how many tokens we are using in each call. This is a good practice that is increasingly important as we use more complex tools that might make several calls to the API (like agents). It is very important to have a close control of how many tokens we are spending to avoid unsuspected expenditures.

우리가 사용할 추가 유틸리티로서 각 호출에서 사용하는 토큰 수를 알려주는  함수가 필요하다. 우리가 API를 여러 번 호출하는 복잡한 도구를 사용해야 하기 때문에  중요하고 좋은 방법이다. 즉, 예상치 못한 비용을 막기 위해 얼마나 많은 토큰을 사용하고 있는지를 항상 감시하는 것이 필요하다.

In [6]:
# 사용한 Token의 수를 카운트...

def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

## What are chains anyway?

**Definition**: Chains are one of the fundamental building blocks of this lib (as you can guess!).

The official definition of chains is the following:


> A chain is made up of links, which can be either primitives or other chains. Primitives can be either prompts, llms, utils, or other chains.


So a chain is basically a pipeline that processes an input by using a specific combination of primitives. Intuitively, it can be thought of as a 'step' that performs a certain set of operations on an input and returns the result. They can be anything from a prompt-based pass through a LLM to applying a Python function to an text.

Chains are divided in three types: Utility chains, Generic chains and Combine Documents chains. In this edition, we will focus on the first two since the third is too specific (will be covered in due course).

1. Utility Chains: chains that are usually used to extract a specific answer from a llm with a very narrow purpose and are ready to be used out of the box.
2. Generic Chains: chains that are used as building blocks for other chains but cannot be used out of the box on their own.

Let's take a peek into what these chains have to offer!

###Utility Chains
Let's start with a simple utility chain. The LLMMathChain gives llms the ability to do math. Let's see how it works!

##Chain의 공식적인 정의는 다음과 같다::::
Chain은 연결로 이루어지며, 이 연결은 프리미티브 또는 다른 Chains일 수도 있다. 프리미티브는 prompts, lllms, utils 또는 다른 chains 일 수 있다.
Chain은 기본적으로 프리미티브들의 특정한 조합을 이용하여 입력을 처리하는 'pipeline' 이다.
직관적으로 입력에 대해 일정한 연산을 수행하고 그 결과를 반환하는 '단계'로 생각할 수 있다. Prompt의 LLM 통과에서부터 텍스트에 파이썬 함수를 적용하는 것까지 모두 가능하다.

## Chain의 종류
Chain은 Utility chain, Generic chain 및 Combine Documents chain의 세 가지 유형으로 구분된다.  세 번째는 복잡하니 설명을 유보한다.

Utility chains : 특정한 목적으로 전체에서 특정한 답을 추출하는 데 사용되는 체인으로 쉽게 사용할 수 있다.
Generic chain: 다른 chain의 구성 요소로 사용되지만 독립적으로는 사용하지 않는 체인이다.


###간단한 Utility chain 부터 시작해보자.
 LLMMath Chain은 LLM들에게 계산을 할 수 있는 능력을 제공한다. 이것을 어떻게 사용하는지 살펴 보자!


#### Pro-tip: use `verbose=True` to see what the different steps in the chain are!
프로 팁: 자세한 내용은 "verbose=True"를 사용하여 체인의 여러 단계가 무엇인지 확인하시오!

In [8]:
llm_math = LLMMathChain(llm=llm, verbose=True)  # 출력을 확인하기 위해 verbose를 사용함

count_tokens(llm_math, "What is 13 raised to the .3432 power?") # 13의 0.3432 제곱은 ?

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm_math/base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(




> Entering new LLMMathChain chain...
What is 13 raised to the .3432 power?
```text
13**.3432
```
...numexpr.evaluate("13**.3432")...

Answer: 2.4116004626599237
> Finished chain.
Spent a total of 264 tokens


'Answer: 2.4116004626599237'

Let's see what is going on here. The chain recieved a question in natural language and sent it to the llm. The llm returned a Python code which the chain compiled to give us an answer. A few questions arise.. How did the llm know that we wanted it to return Python code?

여기서 무슨 일이 일어나고 있는지 보자. Chain은 자연어 질문을 받아서 LLM에 보냈다... LLM은 답을 주기 위해 Chain이 컴파일한 파이썬 code를 반환했다.
여기서 몇 가지 질문이 발생한다. 우리가 파이썬 코드를 반환하기를 원한다는 것을 LLM은 어떻게 알았을까?

The question we send as input to the chain is not the only input that the llm recieves 😉. The input is inserted into a wider context, which gives precise instructions on how to interpret the input we send. This is called a _prompt_. Let's see what this chain's prompt is!

##우리가 Chain에 입력하는 질문은 lllm이 수신하는 입력만이 아니다... 입력은 그 보다 더 넓은 context에 삽입되며,,, context 는 우리가 보내는 입력을 어떻게 해석하는지에 대한  instructions 를 제공한다... 이것을 'Prompt'라고 부른다. 이 Chain의 (감추어진) 프롬프트가 무엇인지 살펴보자!

In [ ]:
print(llm_math.prompt.template)

You are GPT-3, and you can't do math.

You can do basic math, and your memorization abilities are impressive, but you can't do any complex calculations that a human could not do in their head. You also have an annoying tendency to just make up highly specific, but wrong, answers.

So we hooked you up to a Python 3 kernel, and now you can execute code. If anyone gives you a hard math problem, just use this format and we’ll take care of the rest:

Question: ${{Question with hard calculation.}}
```python
${{Code that prints what you need to know}}
```
```output
${{Output of your code}}
```
Answer: ${{Answer}}

Otherwise, use this simpler format:

Question: ${{Question without hard calculation}}
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?

```python
print(37593 * 67)
```
```output
2518731
```
Answer: 2518731

Question: {question}



Ok.. let's see what we got here. So, we are literally telling the llm that for complex math problems **it should not try to do math on its own** but rather it should print a Python code that will calculate the math problem instead. Probably, if we just sent the query without any context, the llm would try (and fail) to calculate this on its own. Wait! This is testable.. let's try it out! 🧐

그럼, 우리가 뭘 얻었는지 보자. 그래서 우리는 LLM에게 복잡한 수학 문제는 수학 계산 자체를 시도할 것이 아니라 수학 문제를 계산하는 파이썬 코드를 출력해야 한다고 문자 그대로 말하고 있다. 아마 우리가 아무런 context 없이 query를 보낸다면 LLM은 스스로 계산하려고 시도하고 (실패할)할 것이다.
---> 이것은 확인 가능하다. 시도해보자!

In [ ]:
# Prompt가 우리가 한 질문을 그대로 나타내게 설정하면... we set the prompt to only have the question we ask

prompt = PromptTemplate(input_variables=['question'], template='{question}')
llm_chain = LLMChain(prompt=prompt, llm=llm)

# we ask the llm for the answer with no context

count_tokens(llm_chain, "What is 13 raised to the .3432 power?")

Spent a total of 17 tokens


'\n\n2.907'

Wrong answer! Herein lies the power of prompting and one of our most important insights so far:

**Insight**: _by using prompts intelligently, we can force the llm to avoid common pitfalls by explicitly and purposefully programming it to behave in a certain way._

오답이다 !!! 여기에 Prompting의 힘과 지금까지 우리가 가졌던 가장 중요한 통찰 중의 하나가 있다:
Insight : 프롬프트를 지능적으로 사용하여, 언어 모델이 특정 방식으로 행동하도록 명시적이고 목적을 가진 프로그래밍을 하게함으로써 일반적인 함정을 피하도록 강제할 수 있다.


Another interesting point about this chain is that it not only runs an input through the llm but it later compiles Python code. Let's see exactly how this works.


이 체인의 또 다른 흥미로운 점은 LLM을 통해 입력을 실행할 뿐만 아니라 나중에 파이썬 코드를 컴파일한다는 것이다. 이것이 정확히 어떻게 작동하는지 보자.

In [9]:
print(inspect.getsource(llm_math._call))

    def _call(
        self,
        inputs: Dict[str, str],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        _run_manager.on_text(inputs[self.input_key])
        llm_output = self.llm_chain.predict(
            question=inputs[self.input_key],
            stop=["```output"],
            callbacks=_run_manager.get_child(),
        )
        return self._process_llm_result(llm_output, _run_manager)



So we can see here that if the llm returns Python code we will compile it with a Python REPL* simulator. We now have the full picture of the chain: either the llm returns an answer (for simple math problems) or it returns Python code which we compile for an exact answer to harder problems. Smart!

Also notice that here we get our first example of **chain composition**, a key concept behind what makes langchain special. We are using the `LLMMathChain` which in turn initializes and uses an `LLMChain` (a 'Generic Chain') when called. We can make any arbitrary number of such compositions, effectively 'chaining' many such chains to achieve highly complex and customizable behaviour.

Utility chains usually follow this same basic structure: there is a prompt for constraining the llm to return a very specific type of response from a given query. We can ask the llm to create SQL queries, API calls and even create Bash commands on the fly 🔥

The list continues to grow as langchain becomes more and more flexible and powerful so we encourage you to [check it out](https://langchain.readthedocs.io/en/latest/modules/chains/utility_how_to.html) and tinker with the example notebooks that you might find interesting.

*_A Python REPL (Read-Eval-Print Loop) is an interactive shell for executing Python code line by line_

여기서 LLM이 파이썬 코드를 반환하면 파이썬 REPL* 시뮬레이터로 컴파일한다는 것을 알 수 있다. 이제 체인의 전체 그림이 완성되었다. LLM이 (단순 수학 문제의 경우) 답을 반환하거나 더 어려운 문제에 대한 정확한 답을 위해 컴파일도하는 파이썬 코드를 반환한다. 똑똑하다!

또한 여기서 우리는 LangChain을 특별하게 만드는 개념인 chain 구성의 첫 번째 예를 얻을 수 있다. 우리는 LLMMathChain을 사용하고 있으며, 이는 호출될 때 LLMChain('일반 체인')을 초기화하고 사용한다. 우리는 임의의 수의 그러한 구성을 만들 수 있으며, 매우 복잡하고 사용자 별로 지정 가능한 동작을 달성하기 위해  많은 Chain을 효과적으로 'chaining'할 수 있습니다.

Utility Chain들은 보통 동일한 기본 구조를 따른다: 주어진 query 로부터 매우 특정한 유형의 응답을 반환하도록 lllm을 제한하는 프롬프트가 있다. 우리는 lllm에게 SQL 쿼리, API 호출, 심지어 Bash 명령어를 즉시 생성하도록 요청할 수 있다.

LangChain이 점점 더 유연해지고 강력해짐에 따라 목록은 계속해서 증가하고 있으므로 여러분이 그것을 확인하고 흥미를 느낄 수 있는 예제 노트를 수정하기 바란다.

*Python REPL(Read-Eval-Print Loop)은 Python 코드를 한 줄씩 실행하기 위한 interactive shell 이다


### Generic chains

There are only three Generic Chains in langchain and we will go all in to showcase them all in the same example. Let's go!

Say we have had experience of getting dirty input texts. Specifically, as we know, llms charge us by the number of tokens we use and we are not happy to pay extra when the input has extra characters. Plus its not neat 😉

First, we will build a custom transform function to clean the spacing of our texts. We will then use this function to build a chain where we input our text and we expect a clean text as output.

LangChain에는 세 개의 generic chain 만 있고 우리는 모두 같은 예로 보여주기 위해 참여할 것이다. 가보자!

우리가 지저분한 문자를 받았다고 가정해보자. 특히, 우리가 알고 있는 바와 같이, LLM은 우리가 사용하는 토큰 수로 우리에게 청구하고 우리는 입력에 추가 문자가 있을 때 추가 비용을 지불하는 것을 좋아하지 않는다. 게다가 그것은 깔끔하지도 않다.

먼저 텍스트의 space 를 제거하기 위해 사용자 정의의 변환 함수를 구축할 것이다. 그런 다음 이 함수를 사용하여 텍스트를 입력하고 기대하는 깨끗한 텍스트를 출력으로하는 체인을 구축할 것이다.

In [10]:
def transform_func(inputs: dict) -> dict:   #  -> 는 함수 리턴 값의 주석 역할을 한다. 즉, 설명일뿐 return 값이 이와 달라도 오류가 생기지 않는다.
    text = inputs["text"]

    # replace multiple new lines and multiple spaces with a single one
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    return {"output_text": text}

Importantly, when we initialize the chain we do not send an llm as an argument. As you can imagine, not having an llm makes this chain's abilities much weaker than the example we saw earlier. However, as we will see next, combining this chain with other chains can give us highly desirable results.

중요한 것은 우리가 Chain을 초기화할 때 llm을 argument로 보내지 않는다는 것이다... 상상하듯이, llm이 없다면 이 Chain의 능력은 앞에서 본 예보다 훨씬 약해진다. 그러나 다음에 보게 되겠지만, 이 Chain을 다른 Chain과 결합하면 매우 바람직한 결과를 얻을 수 있다.

In [11]:
clean_extra_spaces_chain = TransformChain(input_variables=["text"], output_variables=["output_text"], transform=transform_func)

In [13]:
clean_extra_spaces_chain.run('A random text  with   some irregular spacing.\n\n\n     Another one   here as well.')  # space가 많이 포함된 문장

'A random text with some irregular spacing.\n Another one here as well.'

Great! Now things will get interesting.

Say we want to use our chain to clean an input text and then paraphrase the input in a specific style, say a poet or a policeman. As we now know, the `TransformChain` does not use a llm so the styling will have to be done elsewhere. That's where our `LLMChain` comes in. We know about this chain already and we know that we can do cool things with smart prompting so let's take a chance!

우리는 Chain을 사용하여 입력 텍스트를 정리한 다음 특정 스타일(시나 공문서 형태)로 입력을 구문 분석하고 싶다. 우리가 아는 바와 같이 TranformChain 은 lm을 사용하지 않기 때문에 스타일링은 다른 곳에서 해야 할 것이다. 그것이 우리의 LLMChain 이 등장하는 부분이다. 우리는 이미 이 체인에 대해 알고 있고 smart prompting 으로 멋진 일을 할 수 있다는 것을 알고 있으므로 기회를 잡아보자!

First we will build the prompt template:

우선 prompt template를 만들어 보자.

In [14]:
# 참고--- paraphrase= 다른 말로 바꾸어 표현하다.
template = """Paraphrase this text:

{output_text}

In the style of a {style}.

Paraphrase: """
prompt = PromptTemplate(input_variables=["style", "output_text"], template=template)

And next, initialize our chain:

다음은 우리의 chain을 초기화하자.

In [15]:
style_paraphrase_chain = LLMChain(llm=llm, prompt=prompt, output_key='final_output')

Great! Notice that the input text in the template is called 'output_text'. Can you guess why?

We are going to pass the output of the `TransformChain` to the `LLMChain`!

Finally, we need to combine them both to work as one integrated chain. For that we will use `SequentialChain` which is our third generic chain building block.

좋습니다! 템플릿의 입력 텍스트를 'output_text'라고 한다. 그 이유를 추측할 수 있나?

트랜스포트 체인의 출력을 LLMChain에 전달하겠다!

마지막으로 하나의 통합 체인으로 작동하기 위해서는 이 둘을 결합해야 한다. 이를 위해 우리는 세 번째 일반 체인 구축 블록인 Sequential Chain을 사용할 것이다.

In [16]:
sequential_chain = SequentialChain(chains=[clean_extra_spaces_chain, style_paraphrase_chain], input_variables=['text', 'style'], output_variables=['final_output'])

Our input is the langchain docs description of what chains are but dirty with some extra spaces all around.

우리의 input은 어떤 chain들이 여기 저기에 space가 있는 더러운 것인지에 대한 LangChain 문서 설명이다.

In [17]:
input_text = """
Chains allow us to combine multiple


components together to create a single, coherent application.

For example, we can create a chain that takes user input,       format it with a PromptTemplate,

and then passes the formatted response to an LLM. We can build more complex chains by combining     multiple chains together, or by


combining chains with other components.
"""

We are all set. Time to get creative!

다 준비되었다. 창의력을 발휘할 때이다.

In [18]:
count_tokens(sequential_chain, {'text': input_text, 'style': 'a 90s rapper'})

Spent a total of 164 tokens


"\nChains let us link up multiple pieces, makin' one big app. Like, we can take user input, format it with a PromptTemplate, then send it to an LLM. We can get even more complex, combining chains with other components, or just linkin' up multiple chains."

## A note on langchain-hub

`langchain-hub` is a sister library to `langchain`, where all the chains, agents and prompts are serialized for us to use.

langchain-hub 는 langchain의 자매 라이브러리로서, 모든 체인, 에이전트 및 프롬프트가 우리가 사용할 수 있도록 직렬화되어 있다.

In [19]:
from langchain.chains import load_chain

Loading from langchain hub is as easy as finding the chain you want to load in the repository and then using `load_chain` with the corresponding path. We also have `load_prompt` and `initialize_agent`, but more on that later. Let's see how we can do this with our `LLMMathChain` we saw earlier:

langchain hub 로부터 로딩하는 것은 repository에서 로드할 체인을 찾은 다음 해당 경로를 가진 load_chain을 사용하는 것처럼 쉽다. load_prompt와 initialize_agent도 있지만 그 외의 것들도 있다... 앞에서 본 LLMMathChain을 사용하여 이것을 어떻게 할 수 있는지 알아보자

In [20]:
llm_math_chain = load_chain('lc://chains/llm-math/chain.json')

ValidationError: ignored

What if we want to change some of the configuration parameters? We can simply override it after loading:

In [ ]:
llm_math_chain.verbose

True

In [ ]:
llm_math_chain.verbose = False

In [ ]:
llm_math_chain.verbose

False

That's it for this example on chains.

---